## Read the script

In [1]:
import os
import numpy as np
import mne
from ieeg.io import get_data, raw_from_layout, save_derivative
from ieeg.navigate import trial_ieeg, outliers_to_nan, channel_outlier_marker, crop_empty_data
from ieeg.calc.scaling import rescale
from ieeg.viz.ensemble import chan_grid
from ieeg.timefreq.utils import crop_pad, wavelet_scaleogram
from ieeg.viz.parula import parula_map
save_dir='C:\\Users\\bl314\\Box\\CoganLab\\IndividualMeetings\\Baishen\\ieeg_results\\lexical_delay'

In [2]:
HOME = os.path.expanduser("~")
LAB_root = os.path.join(HOME, "Box", "CoganLab")
layout = get_data("LexicalDecRepDelay", root=LAB_root)
subjects = layout.get(return_type="id", target="subject")
subject = 'D0101'

if subject=='D0053':
    subject_Tag = 'D53'
elif subject=='D0054':
    subject_Tag = 'D54'
elif subject=='D0055':
    subject_Tag = 'D55'
elif subject=='D0057':
    subject_Tag = 'D57'
elif subject=='D0059':
    subject_Tag = 'D59'
elif subject=='D0063':
    subject_Tag = 'D63'
elif subject=='D0066':
    subject_Tag = 'D66'
elif subject=='D0069':
    subject_Tag = 'D69'
elif subject=='D0070':
    subject_Tag = 'D70'
elif subject=='D0070':
    subject_Tag = 'D70'
elif subject=='D0081':
    subject_Tag = 'D81'
elif subject=='D0094':
    subject_Tag = 'D94'
elif subject=='D0096':
    subject_Tag = 'D96'
elif subject=='D0101':
    subject_Tag = 'D101'
elif subject=='D0102':
    subject_Tag = 'D102'
elif subject=='D0103':
    subject_Tag = 'D103'
elif subject=='D0107':
    subject_Tag = 'D107B'
else:
    print("Subject not found, please check.")

## Plot subj

In [5]:
from ieeg.viz.mri import plot_subj

# plot the subject brain
fig1 = plot_subj(subject_Tag)
#Label every electrode
mne.viz.set_3d_view(fig1, azimuth=150, elevation=70, focalpoint="auto",
                    distance="auto")

C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\viz\mri.py:696: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)


## Load subj
### <span style="color:red">Please make sure that the **event.tsv** files in the **a** derivatives do not contain **"BAD boundary"** lines, or remove them if they have</span>

In [6]:
raw = raw_from_layout(layout.derivatives['derivatives/a'], subject=subject, desc='a',extension='.edf',preload=True)

Extracting EDF parameters from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\a\sub-D0101\ieeg\sub-D0101_task-LexicalDecRepDelay_acq-01_run-01_desc-a_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\a\sub-D0101\ieeg\sub-D0101_task-LexicalDecRepDelay_acq-01_run-01_desc-a_events.tsv.
Reading channel info from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\a\sub-D0101\ieeg\sub-D0101_task-LexicalDecRepDelay_acq-01_run-01_desc-a_channels.tsv.
Reading electrode coords from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\a\sub-D0101\ieeg\sub-D0101_acq-01_space-ACPC_electrodes.tsv.
Not fully anonymizing info - keeping his_id, sex, and hand info
Extracting EDF parameters from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\a\sub-D0101\ieeg\sub-D0101_task-Lex

C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\io.py:113: RuntimeWarning: Omitted 422 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\io.py:113: RuntimeWarning: The unit for channel(s) Trigger has changed from NA to V.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading events from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\a\sub-D0101\ieeg\sub-D0101_task-LexicalDecRepDelay_acq-01_run-02_desc-a_events.tsv.
Reading channel info from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\a\sub-D0101\ieeg\sub-D0101_task-LexicalDecRepDelay_acq-01_run-02_desc-a_channels.tsv.
Reading electrode coords from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\a\sub-D0101\ieeg\sub-D0101_acq-01_space-ACPC_electrodes.tsv.
Not fully anonymizing info - keeping his_id, sex, and hand info
Extracting EDF parameters from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\a\sub-D0101\ieeg\sub-D0101_task-LexicalDecRepDelay_acq-01_run-03_desc-a_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\io.py:113: RuntimeWarning: Omitted 424 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\io.py:113: RuntimeWarning: The unit for channel(s) Trigger has changed from NA to V.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading events from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\a\sub-D0101\ieeg\sub-D0101_task-LexicalDecRepDelay_acq-01_run-03_desc-a_events.tsv.
Reading channel info from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\a\sub-D0101\ieeg\sub-D0101_task-LexicalDecRepDelay_acq-01_run-03_desc-a_channels.tsv.
Reading electrode coords from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\a\sub-D0101\ieeg\sub-D0101_acq-01_space-ACPC_electrodes.tsv.
Not fully anonymizing info - keeping his_id, sex, and hand info
Extracting EDF parameters from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\a\sub-D0101\ieeg\sub-D0101_task-LexicalDecRepDelay_acq-01_run-04_desc-a_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\io.py:113: RuntimeWarning: Omitted 424 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\io.py:113: RuntimeWarning: The unit for channel(s) Trigger has changed from NA to V.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading events from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\a\sub-D0101\ieeg\sub-D0101_task-LexicalDecRepDelay_acq-01_run-04_desc-a_events.tsv.
Reading channel info from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\a\sub-D0101\ieeg\sub-D0101_task-LexicalDecRepDelay_acq-01_run-04_desc-a_channels.tsv.
Reading electrode coords from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\a\sub-D0101\ieeg\sub-D0101_acq-01_space-ACPC_electrodes.tsv.
Not fully anonymizing info - keeping his_id, sex, and hand info
Reading 0 ... 5066751  =      0.000 ...  2474.000 secs...


C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\io.py:113: RuntimeWarning: Omitted 422 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\io.py:113: RuntimeWarning: The unit for channel(s) Trigger has changed from NA to V.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


## Re-referenced to the average

In [7]:
ch_type = raw.get_channel_types(only_data_chs=True)[0]
raw.set_eeg_reference(ref_channels="average", ch_type=ch_type)

Applying average reference.
Applying a custom ('sEEG',) reference.


<RawEDF | sub-D0101_task-LexicalDecRepDelay_acq-01_run-01_desc-a_ieeg.edf, 214 x 5066752 (2474.0 s), ~8.08 GB, data loaded>

## Wavelet spectrogram

In [8]:
if not os.path.exists(os.path.join(save_dir, subject)):
    os.mkdir(os.path.join(save_dir, subject)) 
if not os.path.exists(os.path.join(save_dir, subject,'wavelet')):
    os.mkdir(os.path.join(save_dir, subject,'wavelet'))

In [9]:
# Wavelet is good to detect and remove muscle artifact channels
# Also plot the subject brain
for task, task_Tag in zip(('Repeat','Yes_No'),('Rep','YN')):
    for epoch, t, tag in zip(
        ('Auditory_stim/'+task+'/CORRECT','Delay/'+task+'/CORRECT','Go/'+task+'/CORRECT','Resp/'+task+'/CORRECT'),
        ((-0.5, 1.5),(-0.5, 1.5),(-0.5, 1.5),(-0.5, 1)),
        ('Auditory-'+task_Tag,'Delay-'+task_Tag,'Go-'+task_Tag,'Resp-'+task_Tag)
    ):
        
        # Get the spectras
        t1 = t[0] - 0.5
        t2 = t[1] + 0.5
        times = (t1, t2)
        trials = trial_ieeg(raw, epoch, times, preload=True)
        outliers_to_nan(trials, outliers=10)
        
        ##############################
        ####### Wavelet ##############
        ##############################
        
        spectra_wavelet = wavelet_scaleogram(trials, n_jobs=-3, decim=int(
            raw.info['sfreq'] / 200)) # 1/10 of the timepionts, don't take too long
        crop_pad(spectra_wavelet, "0.5s") # cut the first and final 0.5s, change to zero
        
        # Get the baseline
        if epoch=='Auditory_stim/'+task+'/CORRECT':
            base_wavelet = spectra_wavelet.copy().crop(-0.5, 0)
            base_wavelet = base_wavelet.average(lambda x: np.nanmean(x, axis=0), copy=True)
        
        # Baseline correction
        spectra_wavelet = spectra_wavelet.average(lambda x: np.nanmean(x, axis=0), copy=True)   
        spectra_wavelet = rescale(spectra_wavelet, base_wavelet, copy=True, mode='ratio')
        spectra_wavelet._data = np.log10(spectra_wavelet._data) * 20
    
        # Save spectras
        filename = os.path.join(save_dir, subject,'wavelet',f'{tag}-tfr.h5')
        mne.time_frequency.write_tfrs(filename, spectra_wavelet, overwrite=True)
        
        # Make spectrograms
        chan_grids=chan_grid(spectra_wavelet, size = (20,10),vlim=(-2, 2), cmap=parula_map)
    
        # Save spectrograms
        fig_count=0
        for fig in chan_grids:
            figdir = os.path.join(save_dir, subject,'wavelet',f'{tag}_{fig_count+1}.jpg')
            chan_grids[fig_count].savefig(figdir,dpi=300)   
            fig_count+=1
            
        # Clean memory
        del spectra_wavelet,filename
  
    del base_wavelet

Used Annotations descriptions: [np.str_('Auditory_stim/Repeat/Nonword/banic/CORRECT'), np.str_('Auditory_stim/Repeat/Nonword/berin/CORRECT'), np.str_('Auditory_stim/Repeat/Nonword/boril/CORRECT'), np.str_('Auditory_stim/Repeat/Nonword/boril/ERR_TASK/REP_YN'), np.str_('Auditory_stim/Repeat/Nonword/caris/CORRECT'), np.str_('Auditory_stim/Repeat/Nonword/cazel/CORRECT'), np.str_('Auditory_stim/Repeat/Nonword/cazel/LATE_RESP'), np.str_('Auditory_stim/Repeat/Nonword/delin/CORRECT'), np.str_('Auditory_stim/Repeat/Nonword/delin/LATE_RESP'), np.str_('Auditory_stim/Repeat/Nonword/devim/CORRECT'), np.str_('Auditory_stim/Repeat/Nonword/devim/LATE_RESP'), np.str_('Auditory_stim/Repeat/Nonword/galed/ERR_TASK/REP_YN'), np.str_('Auditory_stim/Repeat/Nonword/galed/LATE_RESP'), np.str_('Auditory_stim/Repeat/Nonword/galel/CORRECT'), np.str_('Auditory_stim/Repeat/Nonword/gapel/ERR_TASK/REP_YN/LATE_RESP'), np.str_('Auditory_stim/Repeat/Nonword/gapel/LATE_RESP'), np.str_('Auditory_stim/Repeat/Nonword/gorel/

[Parallel(n_jobs=-3)]: Using backend ThreadingBackend with 30 concurrent workers.
[Parallel(n_jobs=-3)]: Done   1 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-3)]: Done  12 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-3)]: Done  25 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-3)]: Done  38 tasks      | elapsed:    8.6s
[Parallel(n_jobs=-3)]: Done  53 tasks      | elapsed:   11.5s
[Parallel(n_jobs=-3)]: Done  68 tasks      | elapsed:   14.5s
[Parallel(n_jobs=-3)]: Done  85 tasks      | elapsed:   16.2s
[Parallel(n_jobs=-3)]: Done 102 tasks      | elapsed:   19.8s
[Parallel(n_jobs=-3)]: Done 121 tasks      | elapsed:   22.4s
[Parallel(n_jobs=-3)]: Done 140 tasks      | elapsed:   25.5s
[Parallel(n_jobs=-3)]: Done 176 out of 213 | elapsed:   31.1s remaining:    6.5s
[Parallel(n_jobs=-3)]: Done 198 out of 213 | elapsed:   33.2s remaining:    2.4s
[Parallel(n_jobs=-3)]: Done 213 out of 213 | elapsed:   35.1s finished
C:\Users\bl314\AppData\Local\Temp\ipykernel_14908\1512640678.py:2

Applying baseline correction (mode: ratio)


C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\calc\scaling.py:67: RuntimeWarning: invalid value encountered in divide
  d /= m


No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No basel

...

No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No basel

[Parallel(n_jobs=-3)]: Using backend ThreadingBackend with 30 concurrent workers.
[Parallel(n_jobs=-3)]: Done   1 tasks      | elapsed:   16.5s
[Parallel(n_jobs=-3)]: Done  12 tasks      | elapsed:   26.4s
[Parallel(n_jobs=-3)]: Done  25 tasks      | elapsed:   27.9s
[Parallel(n_jobs=-3)]: Done  38 tasks      | elapsed:   52.2s
[Parallel(n_jobs=-3)]: Done  53 tasks      | elapsed:   54.6s
[Parallel(n_jobs=-3)]: Done  68 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-3)]: Done  85 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-3)]: Done 102 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-3)]: Done 121 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-3)]: Done 140 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-3)]: Done 176 out of 213 | elapsed:  2.7min remaining:   34.1s
[Parallel(n_jobs=-3)]: Done 198 out of 213 | elapsed:  3.1min remaining:   14.0s
[Parallel(n_jobs=-3)]: Done 213 out of 213 | elapsed:  3.1min finished


Applying baseline correction (mode: ratio)


C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\calc\scaling.py:67: RuntimeWarning: invalid value encountered in divide
  d /= m


No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No basel